In [51]:
#PYSPARK_DRIVER_PYTHON="jupyter" PYSPARK_DRIVER_PYTHON_OPTS="notebook" pyspark
import math
from pyspark import SparkContext, SparkConf

In [52]:
conf = SparkConf().setAppName("Project3").setMaster("local")
try:
    sc.stop()
    sc = SparkContext(conf=conf)
except:
    pass
sc = SparkContext.getOrCreate()

In [53]:
df = sc.textFile("hdfs://localhost/test/input/pto.csv")

In [54]:
def AssignCell(s):
    s = s.split(',')
    x = int(s[0])
    y = int(s[1])
    cell = round(x/20) + round(abs(y-9999)/20)*500+1
    if cell == 1:
        temp = []
        for i in [1, 2, 501, 502]:
            if i == 1:
                temp.append([i,"1,0,3"])
            else:
                temp.append([i,"0,1,0"])
        return temp
    
    elif cell == 249501:
        temp = []
        for i in [249501, 249001, 249002, 249502]:
            if i == 249501:
                temp.append([i,"1,0,3"])
            else:
                temp.append([i,"0,1,0"])
        return temp
    
    elif cell == 500:
        temp = []
        for i in [500, 499, 999, 1000]:
            if i == 500:
                temp.append([i,"1,0,3"])
            else:
                temp.append([i,"0,1,0"])
        return temp
    
    elif cell == 250000:
        temp = []
        for i in [250000, 249499, 249500, 249999]:
            if i == 250000:
                temp.append([i,"1,0,3"])
            else:
                temp.append([i,"0,1,0"])
        return temp
    
    elif (cell>1 and cell<500):
        temp = []
        for i in [cell, cell - 1, cell + 1, cell + 499, cell + 500, cell + 501]:
            if i == cell:
                temp.append([i,"1,0,5"])
            else:
                temp.append([i,"0,1,0"])
        return temp    

    elif (cell>249501 and cell<250000):
        temp = []
        for i in [cell, cell - 1, cell + 1, cell - 499, cell - 500, cell - 501]:
            if i == cell:
                temp.append([i,"1,0,5"])
            else:
                temp.append([i,"0,1,0"])
        return temp  
    
    elif (cell % 500 == 1):
        temp = []
        for i in [cell, cell - 500, cell - 499, cell + 1, cell + 500, cell + 501]:
            if i == cell:
                temp.append([i,"1,0,5"])
            else:
                temp.append([i,"0,1,0"])
        return temp      
    
    elif (cell % 500 == 0):
        temp = []
        for i in [cell, cell - 500, cell - 501, cell - 1, cell + 500, cell + 499]:
            if i == cell:
                temp.append([i,"1,0,5"])
            else:
                temp.append([i,"0,1,0"])
        return temp      
        
    else:
        temp = []
        for i in [cell, cell - 1, cell + 1, cell - 501, cell - 500, cell - 499, cell + 501, cell + 500, cell + 499]:
            if i == cell:
                temp.append([i,"1,0,8"])
            else:
                temp.append([i,"0,1,0"])
        return temp      

In [55]:
def SumCell(s1, s2):
    s1 = s1.split(",")
    s2 = s2.split(",")
    self_cell = int(s1[0]) + int(s2[0])
    relative_cel = int(s1[1]) + int(s2[1])
    relative_cell_count = 0
    if int(s1[2]) != 0:
        relative_cell_count = int(s1[2])
    elif int(s2[2]) != 0:
        relative_cell_count = int(s2[2])
    return str(self_cell)+","+str(relative_cel)+","+str(relative_cell_count)

In [56]:
Coor = df.flatMap(AssignCell)
Sum_Cell = Coor.reduceByKey(SumCell)

In [57]:
def CalDensity(s):
    cell = s[0]
    info = s[1].split(',')
    density = float(info[0]) * float(info[2]) / float(info[1])
    return [int(cell),density]

In [58]:
idx = Sum_Cell.map(CalDensity)
ans = idx.sortBy(lambda x:x[1], ascending=False)

In [59]:
ans = ans.take(50)
density = {}
for n in ans:
    density[n[0]] = n[1]
Top_density = sc.broadcast(density)

In [66]:
def Neighbor_density(s):
    cell = int(s[0])
    density = float(s[1])
    
    if cell == 1:
        temp = []
        for i in [2, 501, 502]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp
    
    elif cell == 249501:
        temp = []
        for i in [249001, 249002, 249502]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp
    
    elif cell == 500:
        temp = []
        for i in [499, 999, 1000]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp
    
    elif cell == 250000:
        temp = []
        for i in [249499, 249500, 249999]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp   
    
    elif (cell>1 and cell<500):
        temp = []
        for i in [cell - 1, cell + 1, cell + 499, cell + 500, cell + 501]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp
    
    elif (cell>249501 and cell<250000):
        temp = []
        for i in [cell - 1, cell + 1, cell - 499, cell - 500, cell - 501]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp
    
    elif (cell % 500 == 1):
        temp = []
        for i in [cell - 500, cell - 499, cell + 1, cell + 500, cell + 501]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])   
        return temp
    
    elif (cell % 500 == 0):
        temp = []
        for i in [cell - 500, cell - 501, cell - 1, cell + 500, cell + 499]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp
    
    else:
        temp = []
        for i in [cell - 1, cell + 1, cell - 501, cell - 500, cell - 499, cell + 501, cell + 500, cell + 499]:
            if i in Top_density.value:
                temp.append([i,str(cell)+","+str(density)])
        return temp

In [69]:
def Aggergate_Neighbor(s1, s2):
    return s1 + "," + s2

In [70]:
neighbor_density = idx.flatMap(Neighbor_density)
neighbor_density = neighbor_density.reduceByKey(Aggergate_Neighbor)

In [71]:
neighbor_density.collect()

[(99975,
  '99474,0.0,99475,0.0,99476,0.470588235294,99974,0.0,99976,1.23076923077,100474,0.470588235294,100475,0.571428571429,100476,0.0'),
 (96136,
  '95635,0.380952380952,95636,0.380952380952,95637,0.0,96135,0.0,96137,1.6,96635,0.842105263158,96636,0.8,96637,0.380952380952'),
 (133001,
  '132501,0.0,132502,0.888888888889,133002,0.444444444444,133501,0.714285714286,133502,0.0'),
 (200459,
  '199958,0.5,199959,0.0,199960,0.0,200458,0.888888888889,200460,1.06666666667,200958,0.0,200959,0.0,200960,0.533333333333'),
 (135052,
  '134551,0.363636363636,134552,0.941176470588,134553,0.0,135051,1.06666666667,135053,0.0,135551,0.421052631579,135552,0.0,135553,0.4'),
 (102158,
  '101657,0.0,101658,0.5,101659,0.347826086957,102157,0.0,102159,0.0,102657,0.0,102658,0.727272727273,102659,0.347826086957'),
 (190737,
  '190236,0.0,190237,0.0,190238,0.0,190736,0.0,190738,0.0,191236,0.0,191237,0.0,191238,0.571428571429'),
 (28309,
  '27808,0.0,27809,0.0,27810,0.0,28308,0.666666666667,28310,1.2307692307